In [18]:
from entities import *
from constants import *
from simulations import *
from tournament_roster import *

import pandas as pd
pd.set_option('display.max_columns', None)
import pyarrow.parquet as pq

from neo4j import GraphDatabase
from tqdm import tqdm 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Full Graph Database

In [2]:
from dataclasses import dataclass

@dataclass
class TeamNode:
    team_id: int
    school: str
    team_name: str
    abbreviation: str
    logo: str
    color: str
    wins: int
    losses: int

@dataclass
class PlayerNode:
    player_id: int
    player_name: str
    player_photo: str
    player_position: str
    player_number: int
    mins_played: int
    team_id: int

In [3]:
from pathlib import Path
team_data = Path.cwd() / 'data' / 'hoopR' / 'team_box_2026.parquet'
player_data = Path.cwd() / 'data' / 'hoopR' / 'player_box_2026.parquet'

## Adding Teams to Memgraph Database

In [4]:
table = pq.read_table(team_data)

# Convert the Arrow Table to a pandas DataFrame
teams_df = table.to_pandas()
teams_df.head()

,game_id,season,season_type,game_date,game_date_time,team_id,team_uid,team_slug,team_location,team_name,team_abbreviation,team_display_name,team_short_display_name,team_color,team_alternate_color,team_logo,team_home_away,team_score,team_winner,assists,blocks,defensive_rebounds,fast_break_points,field_goal_pct,field_goals_made,field_goals_attempted,flagrant_fouls,fouls,free_throw_pct,free_throws_made,free_throws_attempted,largest_lead,lead_changes,lead_percentage,offensive_rebounds,points_in_paint,steals,team_turnovers,technical_fouls,three_point_field_goal_pct,three_point_field_goals_made,three_point_field_goals_attempted,total_rebounds,total_technical_fouls,total_turnovers,turnover_points,turnovers,opponent_team_id,opponent_team_uid,opponent_team_slug,opponent_team_location,opponent_team_name,opponent_team_abbreviation,opponent_team_display_name,opponent_team_short_display_name,opponent_team_color,opponent_team_alternate_color,opponent_team_logo,opponent_team_score
0,401829220,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,2539,s:40~l:41~t:2539,san-francisco-dons,San Francisco,Dons,SF,San Francisco Dons,San Francisco,005a36,ffffff,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,away,73,False,15,1,24,9,46.0,26,56,0,14,65.0,15,23,7,6,30,12,36,4,0,0,32.0,6,19,36,0,14,5,14,2541,s:40~l:41~t:2541,santa-clara-broncos,Santa Clara,Broncos,SCU,Santa Clara Broncos,Santa Clara,690b0b,101010,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,88
1,401829220,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,2541,s:40~l:41~t:2541,santa-clara-broncos,Santa Clara,Broncos,SCU,Santa Clara Broncos,Santa Clara,690b0b,101010,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,home,88,True,17,0,22,21,51.0,33,65,0,20,77.0,10,13,18,6,61,9,38,12,1,0,46.0,12,26,31,0,7,25,7,2539,s:40~l:41~t:2539,san-francisco-dons,San Francisco,Dons,SF,San Francisco Dons,San Francisco,005a36,ffffff,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,73
2,401825477,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,26,s:40~l:41~t:26,ucla-bruins,UCLA,Bruins,UCLA,UCLA Bruins,UCLA,2774ae,f2a900,https://a.espncdn.com/i/teamlogos/ncaa/500/26.png,away,73,True,10,4,28,10,40.0,23,58,0,13,87.0,20,23,20,1,94,12,30,4,0,0,39.0,7,18,40,0,4,11,4,2483,s:40~l:41~t:2483,oregon-ducks,Oregon,Ducks,ORE,Oregon Ducks,Oregon,00934b,fff41b,https://a.espncdn.com/i/teamlogos/ncaa/500/248...,57
3,401825477,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,2483,s:40~l:41~t:2483,oregon-ducks,Oregon,Ducks,ORE,Oregon Ducks,Oregon,00934b,fff41b,https://a.espncdn.com/i/teamlogos/ncaa/500/248...,home,57,False,9,2,25,7,36.0,21,59,0,17,67.0,6,9,2,1,3,11,24,2,2,0,28.0,9,32,36,0,10,8,10,26,s:40~l:41~t:26,ucla-bruins,UCLA,Bruins,UCLA,UCLA Bruins,UCLA,2774ae,f2a900,https://a.espncdn.com/i/teamlogos/ncaa/500/26.png,73
4,401830830,2026,2,2026-01-28,2026-01-28 22:30:00-05:00,36,s:40~l:41~t:36,colorado-state-rams,Colorado State,Rams,CSU,Colorado State Rams,Colorado St,004c23,c8c372,https://a.espncdn.com/i/teamlogos/ncaa/500/36.png,away,50,False,11,1,13,3,33.0,16,48,0,13,71.0,10,14,6,3,10,11,14,5,3,0,27.0,8,30,24,0,17,16,17,21,s:40~l:41~t:21,san-diego-state-aztecs,San Diego State,Aztecs,SDSU,San Diego State Aztecs,San Diego St,a6192e,000000,https://a.espncdn.com/i/teamlogos/ncaa/500/21.png,73


In [5]:
unique_teams = teams_df['team_id'].unique()
team_dfs = [teams_df[teams_df['team_id'] == tid] for tid in unique_teams]

team_nodes = []

for team_df in tqdm(team_dfs):
    row = team_df.iloc[0]
    team_id = int(row[['team_id']])
    school = str(row['team_location'])
    team_name = str(row['team_name'])
    abb = str(row['team_abbreviation'])
    logo = str(row['team_logo'])
    team_color = "#" + str(row['team_color'])
    wins = int(team_df['team_winner'].sum())
    losses = int(len(team_df) - team_df['team_winner'].sum())

    node = TeamNode(
        team_id=team_id,
        school=school,
        team_name=team_name,
        abbreviation=abb,
        logo=logo,
        color=team_color,
        wins=wins,
        losses=losses
    )

    team_nodes.append(node)


100%|██████████| 727/727 [00:00<00:00, 1715.94it/s]


In [6]:
from gqlalchemy import Memgraph
from collections import namedtuple
from dataclasses import asdict

data_for_db = []
for t in team_nodes:
    t_dict = asdict(t) 
    data_for_db.append(t_dict)

memgraph = Memgraph(host="127.0.0.1", port=7687)

query = """
    UNWIND $data AS row
    MERGE (t:Team {id: row.team_id})
    SET t.school = row.school,
        t.name = row.team_name,
        t.abbr = row.abbreviation,
        t.logo = row.logo,
        t.color = row.color,
        t.wins = row.wins,
        t.losses = row.losses
    RETURN count(t) as inserted_teams
"""

results = list(memgraph.execute_and_fetch(query, parameters={"data": data_for_db}))

for row in results:
    print(f"Inserted: {row['inserted_teams']} teams")

Inserted: 727 teams


In [7]:
cols = ['team_id', 'team_score', 'opponent_team_id', 'opponent_team_score', 'team_winner']
teams_df[cols].head()

,team_id,team_score,opponent_team_id,opponent_team_score,team_winner
0,2539,73,2541,88,False
1,2541,88,2539,73,True
2,26,73,2483,57,True
3,2483,57,26,73,False
4,36,50,21,73,False


In [8]:
games_df = teams_df[teams_df['team_winner'] == True].copy()
games_df['score_delta'] = games_df['team_score'] - games_df['opponent_team_score']

games_data = []
for index, row in games_df.iterrows():
    games_data.append({
        "winner_id": int(row['team_id']),
        "loser_id": int(row['opponent_team_id']),
        "delta": int(row['score_delta']),
        "winner_score": int(row['team_score']),
        "loser_score": int(row['opponent_team_score'])
    })

query = """
    UNWIND $data AS row
    MATCH (w:Team {id: row.winner_id})
    MATCH (l:Team {id: row.loser_id})
    MERGE (w)-[r:DEFEATED]->(l)
    SET r.score_delta = row.delta,
        r.winner_score = row.winner_score,
        r.loser_score = row.loser_score
"""

memgraph.execute(query, parameters={"data": games_data})
print(f"Successfully inserted {len(games_data)} game edges.")

Successfully inserted 4087 game edges.


## Adding Players to Memgraph Database

In [9]:
table = pq.read_table(player_data)
player_df = table.to_pandas()
player_df.head()

,game_id,season,season_type,game_date,game_date_time,athlete_id,athlete_display_name,team_id,team_name,team_location,team_short_display_name,minutes,field_goals_made,field_goals_attempted,three_point_field_goals_made,three_point_field_goals_attempted,free_throws_made,free_throws_attempted,offensive_rebounds,defensive_rebounds,rebounds,assists,steals,blocks,turnovers,fouls,points,starter,ejected,did_not_play,active,athlete_jersey,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_display_name,team_uid,team_slug,team_logo,team_abbreviation,team_color,team_alternate_color,home_away,team_winner,team_score,opponent_team_id,opponent_team_name,opponent_team_location,opponent_team_display_name,opponent_team_abbreviation,opponent_team_logo,opponent_team_color,opponent_team_alternate_color,opponent_team_score
0,401829220,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,5176426.0,Junjie Wang,2539,Dons,San Francisco,San Francisco,25.0,3.0,6.0,1.0,4.0,2.0,2.0,1.0,1.0,2.0,0.0,2.0,0.0,2.0,1.0,9.0,True,False,False,True,35,J. Wang,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,San Francisco Dons,s:40~l:41~t:2539,san-francisco-dons,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,SF,005a36,ffffff,away,False,73,2541,Broncos,Santa Clara,Santa Clara Broncos,SCU,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,690b0b,101010,88
1,401829220,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,5174624.0,David Fuchs,2539,Dons,San Francisco,San Francisco,16.0,3.0,3.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,1.0,0.0,0.0,4.0,0.0,7.0,True,False,False,False,8,D. Fuchs,https://a.espncdn.com/i/headshots/mens-college...,Forward,F,San Francisco Dons,s:40~l:41~t:2539,san-francisco-dons,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,SF,005a36,ffffff,away,False,73,2541,Broncos,Santa Clara,Santa Clara Broncos,SCU,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,690b0b,101010,88
2,401829220,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,5176425.0,Ryan Beasley,2539,Dons,San Francisco,San Francisco,31.0,3.0,8.0,2.0,4.0,3.0,4.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0,2.0,11.0,True,False,False,False,0,R. Beasley,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,San Francisco Dons,s:40~l:41~t:2539,san-francisco-dons,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,SF,005a36,ffffff,away,False,73,2541,Broncos,Santa Clara,Santa Clara Broncos,SCU,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,690b0b,101010,88
3,401829220,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,5101799.0,Legend Smiley,2539,Dons,San Francisco,San Francisco,17.0,2.0,3.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,2.0,0.0,0.0,0.0,3.0,4.0,True,False,False,False,13,L. Smiley,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,San Francisco Dons,s:40~l:41~t:2539,san-francisco-dons,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,SF,005a36,ffffff,away,False,73,2541,Broncos,Santa Clara,Santa Clara Broncos,SCU,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,690b0b,101010,88
4,401829220,2026,2,2026-01-28,2026-01-28 23:00:00-05:00,5037875.0,Tyrone Riley IV,2539,Dons,San Francisco,San Francisco,26.0,4.0,17.0,0.0,5.0,0.0,0.0,3.0,5.0,8.0,2.0,0.0,0.0,0.0,1.0,8.0,True,False,False,False,5,T. Riley IV,https://a.espncdn.com/i/headshots/mens-college...,Guard,G,San Francisco Dons,s:40~l:41~t:2539,san-francisco-dons,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,SF,005a36,ffffff,away,False,73,2541,Broncos,Santa Clara,Santa Clara Broncos,SCU,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,690b0b,101010,88


In [10]:
unique_players = player_df['athlete_id'].dropna().unique().astype(int)
player_dfs = [player_df[player_df['athlete_id'] == pid] for pid in unique_players]

In [11]:
player_nodes = []

for p_df in tqdm(player_dfs):
    row = p_df.iloc[0]
    player_id = int(row['athlete_id'])
    player_name = str(row['athlete_display_name'])
    player_photo = str(row['athlete_headshot_href'])
    player_position = str(row['athlete_position_name'])
    player_number = int(row['athlete_jersey']) if row['athlete_jersey'] else None
    mins_played = int(p_df['minutes'].sum())
    team_id = int(row['team_id'])

    node = PlayerNode(
        player_id=player_id,
        player_name=player_name,
        player_photo=player_photo,
        player_position=player_position,
        player_number=player_number,
        mins_played=mins_played,
        team_id=team_id
    )

    player_nodes.append(node)

100%|██████████| 12476/12476 [00:02<00:00, 4628.14it/s]


In [12]:
data_for_db = []
for p in player_nodes:
    p_dict = asdict(p) 
    data_for_db.append(p_dict)

memgraph = Memgraph(host="127.0.0.1", port=7687)

query = """
    UNWIND $data AS row
    MERGE (p:Player {id: row.player_id})
    SET p.player_name = row.player_name,
        p.player_photo = row.player_photo,
        p.player_position = row.player_position,
        p.player_number = row.player_number,
        p.mins_played = row.mins_played,
        p.team_id = row.team_id
    RETURN count(p) as inserted_players
"""

results = list(memgraph.execute_and_fetch(query, parameters={"data": data_for_db}))

for row in results:
    print(f"Inserted: {row['inserted_players']} players")

Inserted: 12476 players


In [13]:
player_data = []
for player in player_nodes:
    player_data.append({
        'player_id': player.player_id,
        'team_id': player.team_id
    })

query = """
    UNWIND $data AS row
    MATCH (p:Player {id: row.player_id})
    MATCH (t:Team {id: row.team_id})
    MERGE (p)-[r:PLAYS_FOR]->(t)
"""

memgraph.execute(query, parameters={"data": player_data})
print(f"Successfully inserted {len(player_data)} player edges.")

Successfully inserted 12476 player edges.


<img src="resources/visualizations/memgraph_database_screenshot.png" width="1000" />

# Creating Simulation Database

In [14]:
import pickle
import itertools
import openpyxl as xl
from constants import *


with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

with open(STAT_EFFECTS, 'rb') as f:
    stats = pickle.load(f)

In [15]:
wb = xl.load_workbook('resources/blank_bracket.xlsx', data_only=True)

tournament_roster = [teams[wb['Teams'][f'C{i}'].value] for i in range(2, 66)]
tournament_roster[:5]

[<Team: Abilene Christian Record: 10-10>,
 <Team: Air Force Record: 3-17>,
 <Team: Akron Record: 17-4>,
 <Team: Alabama Record: 14-6>,
 <Team: Alabama A&M Record: 11-9>]

In [16]:
possible_pairings = list(itertools.combinations(tournament_roster, 2))
possible_pairings[:5]

[(<Team: Abilene Christian Record: 10-10>, <Team: Air Force Record: 3-17>),
 (<Team: Abilene Christian Record: 10-10>, <Team: Akron Record: 17-4>),
 (<Team: Abilene Christian Record: 10-10>, <Team: Alabama Record: 14-6>),
 (<Team: Abilene Christian Record: 10-10>, <Team: Alabama A&M Record: 11-9>),
 (<Team: Abilene Christian Record: 10-10>, <Team: Alabama State Record: 6-14>)]

In [ ]:
tournament_db = []
for t in tournament_roster:
    t_dict = {
        'name': t.name,
        'elo': t.elo,
        'pace': t.pace
        }
    tournament_db.append(t_dict)

memgraph = Memgraph(host="127.0.0.1", port=7687)

query = """
    UNWIND $data AS row
    MERGE (t:Team {id: row.name})
    SET t.elo = row.elo,
        t.pace = row.pace
    RETURN count(t) as inserted_teams
"""

results = list(memgraph.execute_and_fetch(query, parameters={"data": tournament_db}))

for row in results:
    print(f"Inserted: {row['inserted_teams']} teams")

Inserted: 64 teams


In [24]:
games = []

for pairing in tqdm(possible_pairings):
    outcome = simulate_n_games(pairing[0], pairing[1], 3, summary=False)
    games.append(outcome)

100%|█████████▉| 2014/2016 [08:39<00:00, 13.08it/s]

Simulating Game: Creighton vs. Dartmouth


100%|██████████| 2016/2016 [08:39<00:00,  3.88it/s]

Creighton vs. Dartmouth Complete!


In [ ]:
tournament_db = []
for g in games:
    g_dict = {
        'Winner': g['Winner'],
        'Loser': g['Loser'],
        'WinCount': g['Win Count 1']
        }
    tournament_db.append(g_dict)


query = """
    UNWIND $data AS row
    MATCH (w:Team {id: row.Winner})
    MATCH (l:Team {id: row.Loser})
    MERGE (w)-[r:DEFEATED]->(l)
    SET r.wins = row.WinCount
"""

memgraph.execute(query, parameters={"data": tournament_db})
print(f"Successfully inserted {len(tournament_db)} edges.")

Successfully inserted 2016 edges.
